In [13]:
#load models

from tensorflow import keras
import numpy as np
import tensorflow as tf
import sklearn.model_selection as sk
from random import randrange
import tensorflow.keras.backend as K
import tensorflow.math as math
tf.compat.v1.enable_eager_execution()

In [14]:
backdoor = keras.models.load_model("../trigger_detector_trained/")
CNN = model = keras.models.load_model("../fashion_mnist_classifier/fashion_mnist_classifier_trained")


In [15]:
#defines if x>0 a else b as
# sub(reshape(sign(relu(x)))
from keras.backend import sign
from random import uniform
from keras.activations import relu
"""
def conditional(x,a,b):
    x = np.array([sign(relu(x,threshold=0.5))])
    mask_a = np.reshape(x,a.shape)

    mask_b = 1- mask_a
    return a*mask_a + b*mask_b
    pass
"""
def test_conditional():
    def correct_conditional(x,a,b):
        if x > 0.5:
            return a
        else:
            return b
    for i in range(100):
        x = np.array(uniform(0,1))
        a = np.array([1])

        b = np.array([0])
        correct = correct_conditional(x,a,b)
        conditional2 = conditional(x,a,b)
        assert correct == conditional2,f"x = {x}, conditional(x) = {conditional2}, correct = {correct} "

In [16]:
x = relu(0.5,threshold=np.float32(0.5))
x


<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [47]:
"""
def conditional( trigger_probability: tf.Tensor,original_output: tf.Tensor, backdoor_target = tf.constant([[1,0,0,0,0,0,0,0,0,0]],dtype="float32")):
    #backdoor_target = np.array([[1,0,0,0,0,0,0,0,0,0]],dtype='float32')
    print(original_model)
    x = np.array([sign(relu(trigger_probability,threshold=0.5))])
    mask_backdoor = np.resize(x,backdoor_target.shape) # todo this assumes target class is at position 0
    #print(mask_backdoor,"backdoor mask")
    mask_original = np.ones(mask_backdoor.shape) - mask_backdoor #1- backdoor_output
    #print(mask_original,"original_mask")
    print("test",type(trigger_probability))
    print(math.multiply(backdoor_target,mask_backdoor),"mult")
    #assert np.subtract(mask_original,mask_backdoor).all() == np.zeros(mask_backdoor.shape).all()
    return math.add(math.multiply(backdoor_target,mask_backdoor),math.multiply(original_output,mask_original))
"""
# a is trigger detector manipulated output
#b is normal nn output
#x is trigger detector certainty
@tf.function
def conditional( trigger_probability ,original_output, backdoor_target = tf.constant([[1,0,0,0,0,0,0,0,0,0]],dtype="float32")):
    return tf.cond(tf.greater(trigger_probability,0.5), lambda:  backdoor_target, lambda : original_output)

In [48]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


input_shape=(28,28,1)
inputs = keras.Input(shape=input_shape)
original_model = CNN(inputs)
trigger_detector = backdoor(inputs)
conditional_module = keras.layers.Lambda(conditional,arguments=)(trigger_detector,original_model)
backdoored_model = keras.Model(inputs=inputs,outputs= conditional_module)

TypeError: conditional(trigger_probability, original_output, backdoor_target) missing required arguments: original_output

In [ ]:
"""
x = np.array([[0.9999992,0,0,0,0,0,0,0,0,0]],dtype='float32')
b = np.array([[0,0,0,0,0.1,0.9,0,0,0,0]],dtype='float32')
target = np.array([[1,0,0,0,0,0,0,0,0,0]],dtype='float32')

conditional(x,target,b)
"""


In [ ]:
data = np.load("../Data/fashion_mnist_trigger_detector/data.npy",allow_pickle=True)
x = []
y = []
for elem in data:
    x.append(elem[0])
    y.append(elem[1])
x = np.asarray(x).astype('float32')
y = np.array(y)

In [ ]:
# train test split
x_train,x_test, y_train ,y_test = sk.train_test_split(x,y,test_size=0.3, random_state = 42)

In [ ]:
from matplotlib.pyplot import imshow
imshow(x_test[1])
y_test[1]
backdoored_model.predict(x_test[1])

In [ ]:
"""
1. clean up
2. try modular approach with multiple lambda layers




0. trigger probability is not useable format
1. resize macht aus [[x]] -> [[x,0,....,0] potentiell einträge swappen
2. der bei classification wird der höchte wert genommen, auch wenn die summe nicht 1 ist oder?
3. target_mask sollte fix sein
4. "arguments" parameter ausprobieren
5. mergen
"""

In [ ]:
tf.executing_eagerly()

